In [ ]:
import os
import pandas as pd
import numpy as np
from IPython.display import HTML #to display the table with hyperlink
import matplotlib.pyplot as plt

In [ ]:
for dirs in ['mega']:
    main_root=f'/home/thakur/test/{dirs}'
    stocks=os.listdir(main_root)
    #stocks=os.listdir(f'/home/thakur/test/{dirs}')
    print(stocks)

In [ ]:
for k,i in enumerate(stocks):
    if k!=0:continue
    print(f'{k}')
    file=f'{main_root}/{i}'
    print(f'{file}')

In [ ]:
file

In [ ]:
df=pd.read_csv(file)

In [ ]:
#df=df.dropna()
df

In [ ]:
d=df.Date[-1:].values[0][:10]
print(f"LAST DATE: {d}")

In [ ]:
def get_avergevol_past(df,days=10):
    '''
    returns two averge volumes: volumes for days and 50 days past days
    '''
    try:
        head_vol=round(df.tail(50+days).head(50).Volume.mean()/1e6,2)
        tail_vol=round(df.tail(days).Volume.mean()/1e6,2)
        return tail_vol,head_vol
    except:
        print(f"some error in get_averagevol_past!")
        return 0,0

In [ ]:
def get_last_volume(df):
    '''
    retuns the last volume for the given ticker
    '''
    try:
        return round(df.tail(1)['Volume'].values[0]/1e6,2)
    except:
        print(f"some error in get_last_volume!")
        return 0

In [ ]:
def get_last_price(df):
    '''
    retuns the last price for the given ticker
    '''
    try:
        return df.tail(1)['Adj Close'].values[0]
    except:
        print(f"some error in get_last_price !")
        return 0

In [ ]:
def get_second_last_price(df):
    '''
    retuns the last price for the given ticker
    '''
    try:
        return df.tail(2).head(1)['Adj Close'].values[0]
    except:
        print(f"some error get_second_last_price!")
        return 0

In [ ]:
def get_change_pct(df):
    '''
    retuns the last pct_change for the given ticker
    '''
    try:
        return round((get_last_price(df)/get_second_last_price(df)-1)*100,2)
    except:
        print(f"some error in get_change_pct!")
        return 0


In [ ]:
def get_min_price_date(df):
    '''
    retuns the min price and corresponding date for the given interval for the given ticker
    '''
    try:
        idx=df['Adj Close'].idxmin()
        return df['Adj Close'].min(),df.iloc[idx,].Date[:10]
    except:
        print(f"some error get_min_price_date!")
        return 0,0

In [ ]:
def get_max_price_date(df):
    '''
    retuns the max price and corresponding date for the given interval for the given ticker
    '''
    try:
        idx=df['Adj Close'].idxmax()
        return df['Adj Close'].max(),df.iloc[idx,].Date[:10]
    except:
        print(f"some error in get_max_price_date !")
        return 0

In [ ]:
def df_with_link(temp_df):
    '''
    returns a dataframe with the clickable link to tickers for temp_df as the input 
    '''
    df=temp_df
    if 'YAHOO' in list(df.columns):df=df.drop(labels='YAHOO',axis=1)
    df['TICKER'] = df['TICKER'].apply(lambda x: f'<a href="http://finance.yahoo.com/quote/{x}?p={x}&.tsrc=fin-srch">{x}</a>')
    return HTML(df.to_html(escape=False))

In [ ]:
def get_yahoo_link(ticker):
    """
    Get the yahoo link
    """
    #return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,name)
    url="https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch"
    #return '{}" rel="noopener noreferrer" target="_blank">{}'.format(url,ticker)
    return url.format(ticker,ticker)

In [ ]:
def bar_diagram_for_two(df,sel,tit=""):
    '''
    returns a comparion of two selected quantities in a list for given ticker in a given daframe
    '''
    cut=['TICKER']+sel
    df_temp=df[cut]
    labels=list(df['LAST'])
    ax=df_temp.set_index('TICKER').plot(kind='bar',figsize=(20,10))
    ax.bar_label(ax.containers[2],labels=labels, label_type='edge')
    plt.suptitle(f"{tit} [{d}]")
    save_name=f'{tit}.pdf'
    print(f'SAVING AS: {save_name}')
    plt.savefig(save_name)
    plt.show()
    

In [ ]:
def get_all_info(df):
    '''returns the information for the given ticker'''
    #fill na with 0
    #df.fillna(0,inplace=True)
    # df=df.dropna()
    if df.shape[0]<60 or df.empty:
        print(f"rows less than 60!")
        return 
    try:
        last_price=get_last_price(df)
        pct_chg=get_change_pct(df)
        vol=get_last_volume(df)
        mean_5=get_avergevol_past(df,days=5)[0]
        mean_5_50=get_avergevol_past(df,days=5)[1]
        mean_10=get_avergevol_past(df,days=10)[0]
        mean_10_50=get_avergevol_past(df,days=10)[1]
        min_price=get_min_price_date(df)[0]
        min_date=get_min_price_date(df)[1]
        max_price=get_max_price_date(df)[0]
        max_date=get_max_price_date(df)[1]
        return [last_price,pct_chg,vol,min_price,min_date,max_price,max_date,mean_5,mean_5_50,mean_10,mean_10_50]
    except:
        print(f"some error in get_all_info(df) !")
        return 13*[0]
    

In [ ]:

def get_all(dirs):
    '''
    returns dataframe for given category
    '''
    try:
        main_root=f'/home/thakur/test/{dirs}'
        print(f'Working Dir: {main_root}')
        stocks=os.listdir(main_root)
        all_list=[]
        for i in stocks:
            #print(f"Working with: {i}")
            file=f'{main_root}/{i}'
            df=pd.read_csv(file)
            df=df.dropna()
            #df.fillna(0)
            if df.empty:continue
            if df.shape[0]<60:continue
            ticker=i.split('.')[0]
            info_list=[ticker]+get_all_info(df)+[dirs]
            all_list.append(info_list)

        #print(all_list)
        columns=['TICKER', 'LAST', 'CHG%','VOLUME','MIN','MIN_DATE','MAX','MAX_DATE','VOL_5','VOL_5_50','VOL_10','VOL_10_50','CATEGORY']
        return pd.DataFrame(all_list,columns=columns)
    except:
        print(f"some error in get_all(dirs) !")
        return pd.DataFrame()
    
        
    

In [ ]:

def test_rows(dirs):
    '''
    returns dataframe for given category
    '''
    main_root=f'/home/thakur/test/{dirs}'
    print(f'Working Dir: {main_root}')
    stocks=os.listdir(main_root)
    all_list=[]
    for i,k in enumerate(stocks,start=1):
        #print(f"Working with {i}/{len(stocks)}: {k}")
        file=f'{main_root}/{k}'
        df=pd.read_csv(file)
        df=df.dropna()
        if df.empty:
            print(f"{k} is empty!")
            continue
        if df.shape[0]<60:
            print(20*"=")
            print(f"{k} has rows less than 60!")
            continue
        #print(f"WORKING TICKER: {k.split('.')[0]}")
        # ticker=i.split('.')[0]
        # info_list=[ticker]+get_all_info(df)+[dirs]
        # all_list.append(info_list)

    #print(all_list)
    #columns=['TICKER', 'LAST', 'CHG%','VOLUME','MIN','MIN_DATE','MAX','MAX_DATE','VOL_5','VOL_5_50','VOL_10','VOL_10_50','CATEGORY']
    #return pd.DataFrame(all_list,columns=columns)
    
        
    

In [ ]:
dirs='medium'
main_root=f'/home/thakur/test/{dirs}'
print(f'Working Dir: {main_root}')
stocks=os.listdir(main_root)
files=[main_root+"/"+i for i in stocks]

In [ ]:

dirs=['mega','large','medium','small','micro','nano']
fine_list=[]
for m in dirs:
    main_root=f'/home/thakur/test/{m}'
    print(f'Working Dir: {main_root}')
    stocks=os.listdir(main_root)
    files=[main_root+"/"+i for i in stocks]
    for k,i in enumerate(files,start=1):
        df=pd.read_csv(i)
        #print(f"DataFrame: {df}")
        tick=i.split('/')[-1].split('.')[0]
        cat=m
        v=[tick]+get_all_info(df)+[cat]
        fine_list.append(v)
        #print(f"{k:<6}{i.split('/')[-1]:10}\t{v}")

In [ ]:
columns=['TICKER', 'LAST', 'CHG%','VOLUME','MIN','MIN_DATE','MAX','MAX_DATE','VOL_5','VOL_5_50','VOL_10','VOL_10_50','CATEGORY','c0','c1']
cols=['TICKER', 'LAST', 'CHG%','VOLUME','MIN','MIN_DATE','MAX','MAX_DATE','VOL_5','VOL_5_50','VOL_10','VOL_10_50','CATEGORY']
df_fine=pd.DataFrame(fine_list,columns=columns)
df_fine=df_fine[cols]

In [ ]:
df_fine

# SINGLE DATAFRAME

In [ ]:
#df_final
df_fine=df_fine.dropna()
df_fine

In [ ]:
#filter
filter_condition='VOLUME>0.5 & LAST>1.0'
df_fine=df_fine\
.query(filter_condition)\
.reset_index(drop=True)
df_fine

# UP

In [ ]:
up=sum(df_fine['CHG%']>0)
up

# DOWN

In [ ]:
down=sum(df_fine['CHG%']<0)
down

In [ ]:
up_down=pd.DataFrame.from_dict({"UP":[up],"DOWN":[down]})

In [ ]:
up_down

In [ ]:
up_down.plot(kind='bar')

# LAST PRICE < MIN

In [ ]:
df_fine[df_fine['LAST']>df_fine['MIN']].dropna().reset_index(drop=True)

# LAST PRICE = MIN

In [ ]:
df_min=df_fine[df_fine['LAST']==df_fine['MIN']].dropna().reset_index(drop=True).round(2)
#df=df_with_link(df_min)
df_min

# LAST PRICE=MIN; CATEGORY WISE

In [ ]:
grouped=df_min.groupby(['CATEGORY'])
min_dir=df_min.CATEGORY.unique()
for i,cat in enumerate(min_dir,start=1):
    df_temp=grouped.get_group(cat).reset_index(drop=True).sort_values(by='LAST').reset_index(drop=True)
    df_temp.index+=1
    df_temp['y-link']=df_temp['TICKER'].apply(lambda x:get_yahoo_link(x))
    print(f"{i}/{len(dirs)} Working with {dirs[i-1]} ...\n")
    print(f"{df_temp.to_string()}\n")

In [ ]:
# min_dir=list(df_min.CATEGORY.unique())
# grouped=df_min.groupby(['CATEGORY'])
# for i,cat in enumerate(min_dir,start=1):
#     df_temp=grouped.get_group(cat).reset_index(drop=True).sort_values(by='LAST').reset_index(drop=True)
#     if df_temp.empty:continue
#     df_temp['y-link']=df_temp['TICKER'].apply(lambda x:get_yahoo_link(x))
#     print(f"{i}/{len(dirs)} Working with {dirs[i-1]} ...\n")
#     print(f"{df_temp.to_string()}\n")

# LAST PRICE < MAX

In [ ]:
df_fine[df_fine['LAST']<df_fine['MAX']].dropna().reset_index(drop=True)

# LAST PRICE = MAX

In [ ]:
df_max=df_fine[(df_fine['LAST']==df_fine['MAX']) & (df_fine['LAST']>0.0)]\
.dropna()\
.reset_index(drop=True).round(2)\
#.sort_values(by='LAST')\

df_max.head(20)

# LAST_PRICE==MAX; CATEGORIWISE

In [ ]:
grouped=df_max.groupby(['CATEGORY'])
max_dir=df_max.CATEGORY.unique()
for i,cat in enumerate(max_dir,start=1):
    df_temp=grouped.get_group(cat).reset_index(drop=True).sort_values(by='LAST').reset_index(drop=True)
    df_temp.index+=1
    df_temp['y-link']=df_temp['TICKER'].apply(lambda x:get_yahoo_link(x))
    print(f"{i}/{len(dirs)} Working with {dirs[i-1]} ...\n")
    print(f"{df_temp.to_string()}\n")

In [ ]:
# df_maxx=df_with_link(df_max.head(20))
# df_maxx

In [ ]:
df_vol_cat=df_fine.sort_values(by='VOLUME',ascending=False)\
.reset_index(drop=True)

df_vol_head=df_vol_cat.head(30).round(2)
df_vol_head

In [ ]:
df_vol=df_fine.sort_values(by='VOLUME',ascending=False)\
.reset_index(drop=True)

df_vol_head=df_vol.head(30).round(2)
df_vol_head

In [ ]:
# grouped=df_vol.groupby(['CATEGORY'])
# df_vol_micro=grouped.get_group('large')
# df_vol_micro

# MAX VOLUME CATEGORIWISE

In [ ]:
grouped=df_vol.groupby(['CATEGORY'])
for i,cat in enumerate(dirs,start=1):
    df_temp=grouped.get_group(cat).sort_values(by='VOLUME',ascending=False).reset_index(drop=True).head(20).round(2)
    df_temp['y-link']=df_temp['TICKER'].apply(lambda x:get_yahoo_link(x))
    df_temp.index+=1
    print(f"{i}/{len(dirs)}) Working with {dirs[i-1]} ...\n")
    print(f"{df_temp.to_string()}\n")
    bar_diagram_for_two(df_temp,['VOL_10_50','VOL_10','VOLUME'],"MAX_VOLUME-"+cat.upper())

# MAX VOLUME and %CHG>0

In [ ]:
grouped=df_vol.groupby(['CATEGORY'])
for i,cat in enumerate(dirs,start=1):
    df_temp=grouped.get_group(cat).reset_index(drop=True).head(100).round(2)
    df_temp=df_temp[df_temp['CHG%']>0]#.sort_values(by='LAST').reset_index(drop=True)
    if df_temp.empty:continue
    df_temp=df_temp.sort_values(by='VOLUME',ascending=False).reset_index(drop=True)
    df_temp['y-link']=df_temp['TICKER'].apply(lambda x:get_yahoo_link(x))
    df_temp.index+=1
    print(f"{i}/{len(dirs)}) Working with {dirs[i-1]} ...\n")
    print(f"{df_temp.to_string()}\n")
    bar_diagram_for_two(df_temp,['VOL_10_50','VOL_10','VOLUME'],"MAX_VOLUME-CHG%>0-"+cat.upper())

# MAX % GAIN

In [ ]:
df_chg=df_fine.sort_values(by='CHG%',ascending=False)\
.reset_index(drop=True)

df_chg_head=df_chg.head(50).round(2)
df_chg_head

# % CHG CATEGORY WISE

In [ ]:
grouped=df_chg.groupby(['CATEGORY'])
for i,cat in enumerate(dirs,start=1):
    df_temp=grouped.get_group(cat).reset_index(drop=True).head(20).round(2)
    df_temp['y-link']=df_temp['TICKER'].apply(lambda x:get_yahoo_link(x))
    df_temp.index+=1
    print(f"{i}/{len(dirs)}) Working with {dirs[i-1]} ...\n")
    print(f"{df_temp.to_string()}\n")
    bar_diagram_for_two(df_temp,['MIN','MAX','LAST'],"MAX_CHG%-"+cat.upper())

# % MAX LOSS

In [ ]:
df_vol=df_fine.sort_values(by='CHG%',ascending=True)\
.reset_index(drop=True)

df_vol.head(20).round(2)

# % MAX LOSS

In [ ]:
# AVG_VOL_10>AVG_VOL_50
category=['small']#,'medium','large']
df_fine[(df_fine['VOL_10']>df_fine['VOL_10_50']) & (df_fine['CHG%']>0) & (df_fine.CATEGORY.isin(category))].sort_values(by='LAST').reset_index(drop=True)

In [ ]:
codn='(`VOL_10`>`VOL_10_50`) & (`CHG%`>0)'
df_good=df_vol.query(codn).reset_index(drop=True)
df_good.index+=1
df_good

In [ ]:
grouped=df_good.groupby(['CATEGORY'])
for i,cat in enumerate(dirs,start=1):
    df_temp=grouped.get_group(cat).sort_values(by='LAST').reset_index(drop=True).head(20).round(2)
    df_temp['y-link']=df_temp['TICKER'].apply(lambda x:get_yahoo_link(x))
    df_temp.index+=1
    print(f"{i}/{len(dirs)}) Working with {dirs[i-1]} ...\n")
    print(f"{df_temp.to_string()}\n")
    bar_diagram_for_two(df_temp,['MIN','MAX','LAST'],codn+" -"+cat.upper())